In [280]:
import os
import logging

import openai
from langchain_openai import OpenAI
import datasets
from datasets import DatasetDict, load_from_disk
import pandas as pd

logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

model = ChatOpenAI(model="gpt-4o", temperature=0.0)

os.environ["OPENAI_API_KEY"] = ""

In [277]:
# Load dataset
finer = datasets.load_from_disk('hfdata_finer.json')
finer

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3187
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 903
    })
    valid: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 464
    })
})

In [278]:
# Define functions to convert between numeric and text labels
id2label = {
    0: "O",
    1: "B-PER",
    2: "I-PER",
    3: "B-LOC",
    4: "I-LOC",
    5: "B-ORG",
    6: "I-ORG"
}

def convert_id2label(input_list):
    return [id2label.get(item, item) for item in input_list]

label2id = {v: k for k, v in id2label.items()}

def convert_label2id(input_list):
    return [label2id.get(item, item) for item in input_list]

In [310]:
# Set number of examples
k = 64

# Retrieve examples with text labels
examples = []

for i in range(0,k):
    select_example = {}
    select_example["input"] = finer["train"][i]['tokens']
    text_labels = finer["train"][i]['ner_tags']
    select_example["output"] = convert_id2label(text_labels)
    examples.append(select_example)

for i in examples:
    print(i)

{'input': ['Following', 'the', 'closing', 'bell', ',', 'American', 'Express', '(', 'AXP', ')', 'reported', 'weaker', 'earnings', 'and', 'revenue', 'than', 'analysts', 'anticipated', '.'], 'output': ['O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}
{'input': ['All', 'this', 'as', 'the', 'case', 'against', 'him', 'by', 'the', 'receivers', 'for', 'Allco', 'Finance', 'Group', ',', 'the', 'business', 'Fell', 'co-founder', 'with', 'the', 'late', 'David', 'Coe', ',', 'rises', 'again', 'in', 'the', 'Supreme', 'Court', '.'], 'output': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O']}
{'input': ['But', 'growth', 'in', 'spending', 'by', 'companies', 'dealing', 'in', 'business', '-', 'to', '-', 'business', ',', 'or', 'B2B', ',', 'sales', 'was', 'up', 'by', 'nearly', 'twice', 'that', 'at', '61', '%'

In [311]:
from langchain_core.prompts import (ChatPromptTemplate,FewShotChatMessagePromptTemplate)

# Formatting each example as a prompt
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [312]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', """
            You are an expert in Natural Language Processing. 
            Your task is to identify common Named Entities (NER) for each token in a given list.
            Each token can be one of the following labels: ['O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG'].
            Return your answer in the format of a list of labels, where each label corresponds to a token in the input list.
            The number of labels in output sequence should be equal to the number of tokens in the input list.
            Print output list only"""),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

In [313]:
import ast

# Retrieve list of labels from LLM output
def retrieve_labels(output_result):
    res_content = output_result.content
    output_list = ast.literal_eval(res_content)
    return output_list

In [325]:
import time

chain = final_prompt | model

test_results = []

for i in range(0,100):
    print("Predicting", i)
    test_sequence = finer['test']['tokens'][i]
    true_labels = finer['test']['ner_tags'][i]
    res = chain.invoke(test_sequence)
    pred_labels = retrieve_labels(res)
    test_results.append((i, pred_labels))
    # Add sleep to avolide hitting OpenAI rate limit
    time.sleep(15)

Predicting 0


 2024-09-21 10:45:29,080 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
 2024-09-21 10:45:29,092 - INFO - Retrying request to /chat/completions in 0.811299 seconds
 2024-09-21 10:45:33,881 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Predicting 1


 2024-09-21 10:45:51,887 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Predicting 2


...

Predicting 87


 2024-09-21 11:19:18,581 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
 2024-09-21 11:19:18,596 - INFO - Retrying request to /chat/completions in 5.298000 seconds
 2024-09-21 11:19:26,325 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Predicting 88


 2024-09-21 11:19:41,838 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
 2024-09-21 11:19:41,842 - INFO - Retrying request to /chat/completions in 0.978544 seconds
 2024-09-21 11:19:43,023 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
 2024-09-21 11:19:43,026 - INFO - Retrying request to /chat/completions in 1.658556 seconds
 2024-09-21 11:19:44,962 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [315]:
# Show difference in length between input and output sequences
def view_len_results(results_list):
    diff_in_len = []
    
    for i, pred_labels in results_list:
        len_input_sequence = len(finer['test']['tokens'][i])
        diff_in_len.append(len(pred_labels) - len_input_sequence)

    return diff_in_len

print("Difference in length of output and input:", view_len_results(test_results))

Difference in length of output and input: [-1, -3, 0, -8, 3, -1, -4, 3, 1, -1, 0, -1, 0, 1, -2, -1, 0, 0, 0, 0, 0, -3, 4, -1, 0, 0, -1, 0, -1, -1, 0, -2, 0, -4, -1, 0, 0, -2, 0, 0, -1, -2, 5, 0, -2, 0, 0, -2, -1, 1, -2, 0, -1, -1, -3, -1, -2, 2, 2]


In [316]:
# Format list for evaluation
# Match the length of the predicted list with the true list
def match_list(pred_list, true_list):
    while len(pred_list) < len(true_list):
        pred_list.append("O")
    while len(pred_list) > len(true_list):
        pred_list.pop()

    return pred_list

y_true=[]
y_pred=[]

for i in test_results:
    id = i[0]
    pred_labels = i[1]
    true_labels = convert_id2label(finer['test']['ner_tags'][id])
    padded_pred_labels = match_list(pred_labels, true_labels)
    y_true.extend(true_labels)
    y_pred.extend(padded_pred_labels)


In [317]:
from sklearn.metrics import classification_report
print("Classification Report\n")
print(classification_report(y_true, y_pred, labels=['O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG']))

Classification Report

              precision    recall  f1-score   support

           O       0.97      0.96      0.97      1333
       B-PER       0.62      0.67      0.64        12
       I-PER       0.62      0.56      0.59         9
       B-LOC       0.25      0.33      0.29        15
       I-LOC       0.33      0.14      0.20         7
       B-ORG       0.42      0.53      0.47        30
       I-ORG       0.62      0.47      0.53        17

   micro avg       0.94      0.93      0.93      1423
   macro avg       0.55      0.52      0.53      1423
weighted avg       0.94      0.93      0.94      1423



In [318]:
from collections import Counter

# Number of instances in each sequence
y_true_counted = Counter(y_true)
y_pred_counted = Counter(y_pred)

print("Count of y_true instances:", sorted(y_true_counted.items()))
print("Count of y_pred instances:", sorted(y_pred_counted.items()))

Count of y_true instances: [('B-LOC', 15), ('B-ORG', 30), ('B-PER', 12), ('I-LOC', 7), ('I-ORG', 17), ('I-PER', 9), ('O', 1333)]
Count of y_pred instances: [('B-LOC', 20), ('B-MISC', 2), ('B-ORG', 38), ('B-PER', 13), ('I-LOC', 3), ('I-MISC', 1), ('I-ORG', 13), ('I-PER', 8), ('O', 1325)]


In [319]:
print("Predicted labels:", set(y_pred))

Predicted labels: {'I-MISC', 'I-PER', 'I-ORG', 'B-PER', 'B-MISC', 'I-LOC', 'O', 'B-ORG', 'B-LOC'}


In [320]:
def df_from_result(result_tuple):
    df = pd.DataFrame(columns=['ID', 'True Label', 'Predicted Label', 'Token'])
    
    id = result_tuple[0]
    true_labels = convert_id2label(finer['test']['ner_tags'][id])
    pred_labels = match_list(result_tuple[1], true_labels)
    text_sequence = finer['test']['tokens'][id]

    for i in range(len(text_sequence)):
        new_row = (id, true_labels[i], pred_labels[i], text_sequence[i])
        df.loc[len(df)] = new_row
        
    return df

In [321]:
# Convert results to DataFrame format for easy viewing
y_df = pd.DataFrame(columns=['ID', 'True Label', 'Predicted Label'])

for i in test_results:
    result_df = df_from_result(i)
    y_df = pd.concat([y_df, result_df])

# Display all results
pd.set_option('display.max_rows', None)

print(y_df)

    ID True Label Predicted Label               Token
0    0          O               O                   “
1    0          O               O             Overall
2    0          O               O                  it
3    0          O               O             reminds
4    0          O               O                  me
5    0          O               O                  of
6    0          O               O                 the
7    0          O               O                form
8    0          O               O                  of
9    0          O               O        co-operative
10   0          O               O        associations
11   0          O               O                  or
12   0          O               O             mutuals
13   0          O               O                   ,
14   0          O               O                   ”
15   0          O               O                says
16   0          O           B-PER                  Dr
17   0      B-PER           

In [322]:
from datetime import datetime

# Save results to CSV
save_time = datetime.now().strftime("%Y%m%d%H%M")
filename = f"{save_time}_gpt_finer_results.csv"
y_df.to_csv(filename, index=False)